<a href="https://colab.research.google.com/github/kekele-star/fake-news-pred/blob/main/fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>